In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

Adds adjacency to SNG by routing along atlas

In [ ]:
'''
Loading the atlas
'''

atlas = src.graph.graph_from_json('atlas.json')

atlas.number_of_nodes(), atlas.number_of_edges()

In [ ]:
'''
Loading the empty graph
'''

graph_empty = src.graph.graph_from_json('Outputs/graph_empty.json')

graph_empty.number_of_nodes(), graph_empty.number_of_edges()

In [ ]:
'''
Adding road numbers for nodes in empty graph
'''

graph_to_atlas, atlas_to_graph = src.adjacency.node_assignment(graph_empty, atlas)

for graph_node, atlas_node in graph_to_atlas.items():

    graph_empty._node[graph_node]['roadnum'] = atlas._node[atlas_node].get('roadnum', '')

In [ ]:
deep_reload(src)

objective = src.adjacency.Graph_From_Atlas(
    limits = [np.inf, 786e3, np.inf]
)

graph = src.adjacency.adjacency(
    atlas,
    graph_empty,
    objective = objective,
)

In [ ]:
src.graph.graph_to_json(graph, 'Outputs/graph_full.json')

In [ ]:
graph.number_of_nodes(), graph.number_of_edges()

In [ ]:
'''
Loading the graph with adjacency
'''

graph = src.graph.graph_from_json('Outputs/graph_full.json')
graph = src.routing.edge_types(graph)

In [ ]:
'''
Removing non-corridor stations
'''
roadnums = []

for source, node in graph._node.items():
    
    roadnums.append(node['roadnum'])

unique_roadnum, unique_roadnum_count = np.unique(roadnums, return_counts = True)

roadnums_keep = unique_roadnum[(unique_roadnum_count >= 10) & (unique_roadnum != 'nan')]
roadnums_keep = np.append(roadnums_keep, 'U395')

corridor_roadnum = []

corridor_roadnum.extend([r for r in unique_roadnum if 'I' in r])
corridor_roadnum.extend([r for r in unique_roadnum if 'U' in r])
corridor_roadnum.extend(['S1', 'S41', 'S99'])

roadnums_keep = corridor_roadnum

nodes_keep = []

for source, node in graph._node.items():

    roadnum = node['roadnum']

    if node.get('type', '') != 'station':

        node['type'] = 'place'
        nodes_keep.append(source)
    
    else:

        if roadnum in roadnums_keep:
    
            nodes_keep.append(source)

roadnums_keep = np.append(roadnums_keep, 'U395')

sng_corridor = src.graph.subgraph(graph, nodes_keep)

sng_corridor.number_of_nodes(), sng_corridor.number_of_edges()

In [ ]:
'''
Creating Tesla and non-tesla SNGs
'''

u, c = np.unique(
    [n.get('network', '') for n in sng_corridor._node.values()], return_counts = True
)

other_networks = u[(c >= 50) & (u != 'Tesla') & (u != 'Non-Networked')]
other_networks = u

tesla = []
other = []
combined = []
rivian = []

for source, node in sng_corridor._node.items():

    if node['type'] == 'place':

        tesla.append(source)
        other.append(source)
        combined.append(source)
        rivian.append(source)

    else:

        include = False

        if node['network'] == 'Tesla':

            tesla.append(source)
            combined.append(source)

        else:

            other.append(source)
            
        if node['network'] in other_networks:

            combined.append(source)

        if node['network'] in ['RIVIAN_ADVENTURE']:

            rivian.append(source)

sng_tesla = src.graph.subgraph(graph, tesla)
sng_other = src.graph.subgraph(graph, other)
sng_combined = src.graph.subgraph(graph, combined)
sng_rivian = src.graph.subgraph(graph, rivian)

In [ ]:
sng_combined.number_of_nodes(), sng_tesla.number_of_nodes(), sng_other.number_of_nodes(), sng_rivian.number_of_nodes()

In [ ]:
src.graph.graph_to_json(sng_combined, 'Outputs/sng_combined.json')
src.graph.graph_to_json(sng_tesla, 'Outputs/sng_tesla.json')
src.graph.graph_to_json(sng_other, 'Outputs/sng_other.json')
src.graph.graph_to_json(sng_rivian, 'Outputs/sng_sng_rivian.json')